In [ ]:
import timeit
from lxml import html
import requests
import pandas as pd
import numpy as np
start = timeit.default_timer()
resultados=pd.DataFrame()
for id in range(1, 399999): #recorrer los ids de partido
    id=str(id) # convertir id a str para poder concatenar!
    page = requests.get('http://www.espnfc.com/commentary/'+id+'/commentary.html')
    tree = html.fromstring(page.text)
    #listado de minutos
    times = tree.xpath('//div[@class="timestamp"]/p/text()')
    #de todos los div cuya class=timestamp ponme el texto
    #Listado de comments
    comments = tree.xpath('//div[@class="comment"]/p/text()')
    times=[(s[:-1]) for s in times] #esta linea elimina los dos ultimos caracteres de cada elemento del vector
    times2=pd.Series(times)
    times2=times2.convert_objects(convert_numeric=True) #tenemos los tiempos convertidos a numeros desde str!
    comments2=pd.Series(comments)
    corner=comments2.str.startswith('Corner', na=False)#vector true en cada posicion que empieza por "Corner"
    df = pd.DataFrame({'time':times2, 'corner':corner})
    if len(df[(df.corner==True)])!=0:
        minc1=np.amin(df[(df.corner==True)]).time #MINUTO DEL PRIMER CORNER!
        #print minc1 #si no hay corners ese partido no es correcto
        team_away = tree.xpath('//div[@class="team away"]/p/a/text()')
        team_home=tree.xpath('//div[@class="team home"]/p/a/text()')
        hay_corner=minc1<10 #corner antes del 9?
        page = requests.get('http://www.espnfc.com/gamecast/statistics/id/'+id+'/statistics.html')
        tree = html.fromstring(page.text)
        competicion = tree.xpath('//div[@class="match-details"]/p[@class="floatleft"]/text()')
        try:
            competicion= competicion[0].strip()
        except:
            competicion="ERR COMPETICION"
        home_corners = tree.xpath('//td[@id="home-corner-kicks"]/text()')
        away_corners = tree.xpath('//td[@id="away-corner-kicks"]/text()')
        partido=pd.Series([id,str(minc1),str(team_away),str(team_home),str(hay_corner),competicion,home_corners,away_corners])
        resultados=resultados.append(partido,ignore_index=True)
print resultados
#resultados.to_csv("corners.csv",encoding="utf-8")
f = open("corners.csv", 'a') # Añadir los resultados al archivo corners.csv
resultados.to_csv(f, header = False,encoding="utf-8")
f.close()

stop = timeit.default_timer()
print stop - start 